# SANTANDER CUSTOMER TRANSACTION PREDICTION

This is my first Kaggle Notebook where I tried to solve the problem using PCA analysis. 

## Part I - Loading the data and Python libraries[](http://)

### Python libraries

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import csv 
import seaborn as sns
import plotly
import scipy.special
import xgboost as xgb 
import statsmodels.formula.api as smf
from scipy import stats
from scipy.stats import norm, skew
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from sklearn.metrics import roc_curve, roc_auc_score


### Loading the file "sample_submission.csv"

This file is only an example of how to submit the solution to Kaggle.

In [ ]:
df_sample = pd.read_csv(r'/kaggle/input/santander-customer-transaction-prediction/sample_submission.csv')

In [ ]:
df_sample.head()

In [ ]:
df_sample.tail()

In [ ]:
df_sample.target.value_counts()

### Loading file "test.csv"

This is the file that contains the data that our model will use to predict wheter the customer will do or not a specific transaction (or the probability of doing this transaction). 

In [ ]:
df_test_submission = pd.read_csv(r'/kaggle/input/santander-customer-transaction-prediction/test.csv')

In [ ]:
df_test_submission.head()

In [ ]:
list (df_test_submission.columns.values)

In [ ]:
df_test_submission.shape

> ### Loading the "train" file

This is the file that we be used to train our machine learning model.

In [ ]:
df_train0 = pd.read_csv(r'/kaggle/input/santander-customer-transaction-prediction/train.csv')
df_train = df_train0.copy()

## Part II - Pre-processing and EDA (Exploratory Data Analysis)

I will first analyse the train.csv file:

In [ ]:
df_train.shape

In [ ]:
#pd.set_option('display.max_columns', None) # option to display all columns
df_train.head()

We have a quite large number os features and all of them are anonymized. So even if I were an expert in banking, it would still be difficult to reach any conclusion or to formulate hypothesis only by looking at the features. So let's go on with our analysis.

 Verification of null values

In [ ]:
# Looking at other Notebooks at Kaggle, I found this funtion from Gabriel Preda that shows the type of each feature and the percentage of null values in each of them.
#Source: https://www.kaggle.com/gpreda/santander-eda-and-prediction

In [ ]:
def missing_data(data):
    total = data.isnull().sum()
    percent = (data.isnull().sum()/data.isnull().count()*100)
    tt = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    types = []
    for col in data.columns:
        dtype = str(data[col].dtype)
        types.append(dtype)
    tt['Types'] = types
    return(np.transpose(tt))

In [ ]:
%time
missing_data(df_train)

That's good! At least we don't have any missing values in our dataset.

In [ ]:
df_train.info()

In [ ]:
#pd.set_option('display.max_columns', None)
df_train.describe()

Anlysis of the target variable

In [ ]:
df_train['target'].value_counts()

In [ ]:
perc_0 = 100*(sum(df_train['target']==0))/df_train.shape[0]
perc_1 = 100*(sum(df_train['target']==1))/df_train.shape[0]
print ('Percentual de 0 = ',perc_0)
print ('Percentual de 1 = ', perc_1)

In [ ]:
plt.figure(figsize=(3,4))
ax = sns.countplot(x = df_train.target, data = df_train)


In [ ]:
df_train['target'].sample(1000,random_state=42).plot.kde()

We can see that the target variable is imbalanced, as we have a much bigger amount of target equals to zero than equals to 1. We will see later how to balance our data.

### Distribution analysis

In [ ]:
#target_0 = df_train.loc[df_train['target']==0]
#target_1 = df_train.loc[df_train['target']==1]
#features = df_train.columns.values[2:202]

#for i in features:
    #plt.figure(figsize=(4,2))
    #plt.title("Distribution")
    #sns.distplot(target_0[i], kde=True, bins =60, color = 'blue', label = 'target_0')
    #sns.distplot(target_1[i], kde=True, bins =60, color = 'red', label = 'target_1')
    #sns.set(font_scale=0.8)
    #plt.legend()
    #plt.show()

### Scatter Plot

In [ ]:
#var = list (df_train.columns.values)
#var.remove('ID_code')
#var.remove('target')

#sample1 = df_train[df_train['target']==0].sample(500, random_state = 42)
#sample2 = df_train[df_train['target']==1].sample(500, random_state = 42)

# Plot
#for i in var:
    #plt.scatter(y=sample1[i], x=sample1[i].index, alpha=0.4)
    #plt.scatter(y=sample2[i], x=sample2[i].index, alpha=0.4, color = 'red')
    #plt.title(i)
    #plt.xlabel('indice')
    #plt.ylabel(i)
    #plt.show()

### Bloxplot - imbalanced data

In [ ]:
#for i in var:
    #f, ax = plt.subplots(figsize =(3,3))
    #ax = sns.boxplot(x=df_train['target'], y=df_train[i])

### Feature importance analysis

In [ ]:
from sklearn.ensemble import RandomForestClassifier

Declaring X and y

In [ ]:
X_train = df_train.drop(columns =[ 'ID_code','target'], axis=1)

In [ ]:
y_train = df_train['target']

Creating a list with the columns names

In [ ]:
feat_labels = X_train.columns

In [ ]:
feat_labels

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)

rf.fit(X_train, y_train)
importances = rf.feature_importances_

indices = np.argsort(rf.feature_importances_)[::-1]

for f in range(X_train.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30,feat_labels[indices[f]], importances[indices[f]]))

In [ ]:
# importances: returns an array with the importance level of each feature, ordered by the column name 

# indices: return an array with the feature number, according to its importance, descending order 

In [ ]:
indices

### Correlation analysis between independent variables and target

In [ ]:
corrmat = df_train.corr()['target'].sort_values(ascending = False)
corrmat[0:40]                                                        

In [ ]:
# 10 most positive correlated variables with the target

In [ ]:
corrmat = df_train.corr()
cols = corrmat.nlargest(11,'target')['target'].index
cols

In [ ]:
# 10 most negative correlated variables with the target

In [ ]:
cols_negative = corrmat.nsmallest(10,'target')['target'].index
cols_negative

### Heatmap : 20 most correlated variables with the target

In [ ]:
colunas_heat_map = []
for i in cols:
    colunas_heat_map.append (i)
for j in cols_negative:
    colunas_heat_map.append(j)

colunas_heat_map

In [ ]:
corrmat = df_train[colunas_heat_map].corr()
sns.set(font_scale=1.15)
f, ax = plt.subplots(figsize=(20, 20))
hm = sns.heatmap(corrmat, 
                 cbar=True, # formatando a barra lateral de cores para o heatmap
                 annot=True, 
                 square=True, 
                 fmt='.2f', 
                 annot_kws={'size': 8}, 
                 yticklabels=corrmat.columns, 
                 xticklabels=corrmat.columns)

We can notice that even for the most correlated features, the correlation value is quite low.

### Pairplot

I divided the dataset in 20 parts in order to do the pairplot:

In [ ]:
colunas = list(df_train.columns.values)
selected_columns=[]
contador = 1
while contador <= len(colunas):
    selected_columns =  colunas[(contador+1):(contador+12)]
    selected_columns.append('target')
    df1 = df_train[selected_columns].copy()
    sns.pairplot(data = df1.sample(100, random_state =42), height=1.8, hue = 'target')
    contador += 11
    

## Exclusion of extreme values

In [ ]:
# Number of row of the original dataset df_train

qtd_linhas = df_train.shape[0]

In [ ]:
qtd_linhas

In [ ]:
# Function that excludes data that are bellow or above 3*standard deviations from the mean:
def exclui_extremos (nome_variavel):
    global df_train
    x_menos_3dv = df_train[nome_variavel].mean()-(3*df_train[nome_variavel].std())
    x_mais_3dv = df_train[nome_variavel].mean()+(3*df_train[nome_variavel].std())
    linhas_selecionadas = ((df_train[nome_variavel]<x_mais_3dv)&(df_train[nome_variavel]>x_menos_3dv))   
        
    df_train = df_train[linhas_selecionadas].copy()
    
    return 

In [ ]:
colunas = list(df_train.columns.values)

In [ ]:
colunas.remove('ID_code')
colunas.remove('target')

In [ ]:
for i in colunas:
    exclui_extremos(i)

In [ ]:
# New dataset without extreme values:
df_train.shape

In [ ]:
# Percentage of data that has been preserved:

perc_preservado = df_train.shape[0]*100/qtd_linhas
perc_preservado

In [ ]:
# Redefinition of X e y

X_train = df_train.drop(columns =[ 'ID_code','target'], axis=1)

In [ ]:
y_train = df_train['target']

##  Balancing the data : oversampling: SMOTE

In [ ]:
#First, let's separate the data in train and test.

# References that I used:
#https://beckernick.github.io/oversampling-modeling/
#https://imbalanced-learn.readthedocs.io/en/stable/generated/imblearn.over_sampling.SMOTE.html
#https://medium.com/analytics-vidhya/balance-your-data-using-smote-98e4d79fcddb
#https://towardsdatascience.com/a-deep-dive-into-imbalanced-data-over-sampling-f1167ed74b5

The SMOTE oversampling method creates synthetic data based on the observation of points in the neighbourhood (k-neighbors). In comparison with the random-oversampling method, using SMOTE we are less likely to have overfitting.

### Train and test split

In [ ]:
X_train,X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

The argument stratify verifies the proportion between 0 and 1 of the target (y_train) and mantain the same proportion in the train and test .


In [ ]:
X_train.shape

In [ ]:
X_train[0:5]

In [ ]:
y_train.shape

In [ ]:
y_train[0:5]

In [ ]:
X_test.shape

### Balancing of X_train

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
sm = SMOTE(random_state=42, ratio = 1.0)
X_train_bal, y_train_bal = sm.fit_sample(X_train, y_train)

# ratio = 1, means that after the balancing we would like to have the same proportion of 1 and 0 for the target 
# default: k-neighbors = 5

In [ ]:
# Verification of the balanced data

In [ ]:
# X_train_bal:

In [ ]:
X_train_bal.shape

In [ ]:
type(X_train_bal)

In [ ]:
X_train_bal[0:5]

In [ ]:
# y_train_bal:

In [ ]:
y_train_bal.shape

In [ ]:
type(y_train_bal)

In [ ]:
y_train_bal

In [ ]:
# X_train_bal e y_train_bal possuem o mesmo número de linhas (ok)

In [ ]:
# Verficando o número de target = 0 e target =1 

In [ ]:
import collections, numpy
collections.Counter(y_train_bal)

In [ ]:
# Ok, the data is balanced!

In [ ]:
# I renamed the X_train and y_train with the balanced data

In [ ]:
X_train = X_train_bal
y_train = y_train_bal

##  PCA - Principal Component Analysis

As we are dealing with a problem with a high dimensionality and it was quite difficult do define if it's possible to ignore some variables, I will use PCA. 

### Standard Scaler

In [ ]:
#importando a biblioteca
from sklearn.preprocessing import StandardScaler
#instanciando a variável
sc = StandardScaler()
#ajustando os dados de treino
X_train = sc.fit_transform(X_train)
#transformando os dados de teste
X_test = sc.transform(X_test)

In [ ]:
# Agora temos:
# X_train com dados balanceados e na mesma escala
# X_test não balanceado e na mesma escala (aplicamos o scaling)
# y_test não balanceado (sem necessidade de scaling)
# y_train balanceado (sem necessidade de scaling)

### Running PCA

In [ ]:
# importando as bibliotecas
from sklearn.decomposition import PCA


In [ ]:
for i in range (20,51,10):
    print ('Number of components = ', i)
    pca = PCA(n_components = i)
    X_train_pca = pca.fit_transform(X_train)
    print ('*'*30)
    print ('Noise Variance: ', pca.noise_variance_)
    print('Components: ', pca.components_)
    print ('*'*30)

In [ ]:
pca.n_components

In [ ]:
pca.explained_variance_ratio_

In [ ]:
pca.explained_variance_

In [ ]:
X_train[0]

In [ ]:
# Verificando X_train_pca (principal components) após aplicação do PCA 

In [ ]:
X_train_pca[0]

In [ ]:
X_train_pca.shape

In [ ]:
# transformando os dados de teste
X_test_pca = pca.transform(X_test)

In [ ]:
X_test[0]  #primeira linha

In [ ]:
# Verificando X_test_pca (principal components) após aplicação do PCA 

In [ ]:
X_test_pca[0]

In [ ]:
X_test_pca.shape

In [ ]:
# Analisando quais são os "principal components" mais importantes

In [ ]:
print(abs( pca.components_ ))

In [ ]:
pca.components_.shape

In [ ]:
# Primeiro principal component:

In [ ]:
pca.components_[0]

In [ ]:
# Encontrando qual o índice da feature mais importante em cada um dos 50 principal components:

In [ ]:
most_important = [np.abs(pca.components_[i]).argmax() for i in range(50)]

In [ ]:
most_important

In [ ]:
# Identificando o nome das features mais importantes para cada principal component

In [ ]:
# Nome de todas as colunas do df_train

In [ ]:
feature_names = list(df_train.columns.values)

In [ ]:
feature_names[0:5]

In [ ]:
# exluindo o nome das features 'ID_code' e 'target' 

In [ ]:
feature_names.remove('ID_code')

In [ ]:
feature_names.remove('target')

In [ ]:
len(feature_names)

In [ ]:
# Imprimindo o nome da features mais importante para cada principal component

In [ ]:
most_important_names = [feature_names[most_important[i]] for i in range(50)]

In [ ]:
most_important_names

In [ ]:
# References:
#https://stackoverflow.com/questions/50796024/feature-variable-importance-after-a-pca-analysis
# https://github.com/gaurav-kaushik/Data-Visualizations-Medium/blob/master/code/Interactive_PCA_and_Feature_Correlation.ipynb
# https://medium.com/cascade-bio-blog/creating-visualizations-to-better-understand-your-data-and-models-part-1-a51e7e5af9c0

### Estudando a variância dos componentes

In [ ]:
# Variance from reconstructed data
reconstruct = pca.inverse_transform(X_train_pca)
differenceMatrix = X_train - reconstruct
differenceMatrix.var()

# Indicativo de quanta informação foi perdida com o PCA
# Quanto menor, melhor

In [ ]:
reconstruct.shape

In [ ]:
#definindo uma lista com os nomes das colunas
feature_cols = list(df_train.columns)
feature_cols.remove('target')
feature_cols.remove('ID_code')

In [ ]:
# Imagem retirada do Notebook para reduzir o tamanho do arquivo.

In [ ]:
## Feature importance
#plt.figure(figsize=(300,60))
#sns.heatmap(np.log(pca.inverse_transform(np.eye(X_train_pca.shape[1]))), cmap="hot", cbar=False, annot=True, xticklabels=df_train.columns)

## Part III - Modeling

In [ ]:
# Agora vamos atribuir os valores de X_train e X_test com os valores do PCA

In [ ]:
X_train= X_train_pca
X_test = X_test_pca

In [ ]:
# Aplicando o pipeline de marchine learning 

In [ ]:
# ignorando os warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# definindo uma lista com os nomes das colunas  ## feito anteriormente
#feature_cols = list(df_train.columns)
#feature_cols.remove('target')
#feature_cols.remove('ID_code')

# importnado as bibliotecas com os modelos classificadores
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression   
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
# definindo uma lista com todos os classificadores
classifiers = [
    #KNeighborsClassifier(3),
    GaussianNB(),
    LogisticRegression(),
    #SVC(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    GradientBoostingClassifier()]

# definindo o tamanho da figura para o gráfico
plt.figure(figsize=(12,8))

# rotina para instanciar, predizer e medir os resultados de todos os modelos
for clf in classifiers:
    # instanciando o modelo
    clf.fit(X_train, y_train)
    # armazenando o nome do modelo na variável name
    name = clf.__class__.__name__
    # imprimindo o nome do modelo
    print("="*30)
    print(name)
    # imprimindo os resultados do modelo
    print('****Results****')
    y_pred = clf.predict(X_test)
    print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
    print("Precision:", metrics.precision_score(y_test, y_pred))
    print("Recall:", metrics.recall_score(y_test, y_pred))
    
    
    # plotando a curva ROC
    y_pred_proba = clf.predict_proba(X_test)[::,1]
    fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
    auc = metrics.roc_auc_score(y_test, y_pred_proba)
    plt.plot(fpr,tpr,label=name+", auc="+str(auc))
    plt.plot([0, 1], [0, 1], color='red', lw=2, linestyle='--')
    plt.legend(loc=4)

In [ ]:
# Melhores resultados foram com LogisticRegression e GradientBoostingClassifier, 
# porém o valor de precision ainda é bem baixo em ambos os modelos.

### Análise GradientBoostingClassifier

In [ ]:
from sklearn import metrics

In [ ]:
clf

In [ ]:
# atribuindo os valores da matriz às variáveis 
# tp: True Positive
# fp: False Positive
# fn: False Negative
# tn: True Negative

tn, fp, fn, tp = confusion_matrix(y_test,y_pred).ravel()

tn, fp, fn, tp

In [ ]:
# Cálculo da accuracy
# Representa  proporção de casos que foram corretamente previstos, 
#sejam eles verdadeiro positivo ou verdadeiro negativo.

accuracy = (tp+tn)/(tp+fp+tn+fn)
accuracy

In [ ]:
# Cálculo do score de outra forma (accuracy)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

In [ ]:
# Cálculo da precisão

In [ ]:
precision = tp/(tp+fp)
precision

In [ ]:
# Cálculo da Sensiblidade / Recall
# TPR: True Positive rate
# Representa a proporção de casos positivos que foram identificados corretamente.

recall = tp / (tp+fn)
recall

In [ ]:
# Cálculo da specificity
# Representa proporção de casos negativos que foram identificados corretamente

specificity = tn/(tn+fp)
specificity

In [ ]:
# Cálculo da prevalence

prevalence = (tp+fn)/(tp+fn+tn+fp)
prevalence

In [ ]:
# Cálculo do F1 score
f1_gb = 2*(precision*recall)/(precision+recall)
f1_gb

In [ ]:
#taxa de falsos positivos
false_positive = fp/(tn+fp)
false_positive

#### Aplicando o Cross validation para o GradientBoostingClassifier

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, X_train, y_train, cv=5, scoring='accuracy')
list(scores)

In [ ]:
scores.mean()

In [ ]:
mean_scores_GB = scores.mean()

###  LogisticRegression

In [ ]:
# Temos que rodar novamente:

In [ ]:
# Instanciando o modelo:
logreg = LogisticRegression().fit(X_train, y_train)

In [ ]:
y_pred = logreg.predict(X_test)
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

In [ ]:
# atribuindo os valores da matriz às variáveis 
# tp: True Positive
# fp: False Positive
# fn: False Negative
# tn: True Negative

tn, fp, fn, tp = confusion_matrix(y_test,y_pred).ravel()

tn, fp, fn, tp

In [ ]:
# Cálculo da accuracy
# Representa  proporção de casos que foram corretamente previstos, 
#sejam eles verdadeiro positivo ou verdadeiro negativo.

accuracy = (tp+tn)/(tp+fp+tn+fn)
accuracy

In [ ]:
# Cálculo do score de outra forma (accuracy)

In [ ]:
accuracy_score(y_test,y_pred)

In [ ]:
# Cálculo da precisão

In [ ]:
precision = tp/(tp+fp)
precision

In [ ]:
# Cálculo da Sensiblidade / Recall
# TPR: True Positive rate
# Representa a proporção de casos positivos que foram identificados corretamente.

recall = tp / (tp+fn)
recall

In [ ]:
# Cálculo da specificity
# Representa proporção de casos negativos que foram identificados corretamente

specificity = tn/(tn+fp)
specificity

In [ ]:
# Cálculo da prevalence

prevalence = (tp+fn)/(tp+fn+tn+fp)
prevalence

In [ ]:
# Cálculo do F1 score

f1 = 2*(precision*recall)/(precision+recall)
f1

In [ ]:
#taxa de falsos positivos
false_positive = fp/(tn+fp)
false_positive

#### Aplicando o Cross validation para LogisticRegression

In [ ]:
scores_logreg = cross_val_score(logreg, X_train, y_train, cv=5, scoring='accuracy')
list(scores_logreg)

In [ ]:
mean_scores_logreg= scores_logreg.mean() 

In [ ]:
mean_scores_logreg

## Definition of the target and submission to Kaggle

In [ ]:
# Data for submission

In [ ]:
df_test_submission.head()

In [ ]:
df_test_submission.shape

In [ ]:
# Primeiro temos que definir X_test_submission

In [ ]:
X_test_submission = df_test_submission.drop(columns =[ 'ID_code']).copy()

In [ ]:
# Verificando a remoção da coluna

In [ ]:
X_test_submission.head()

In [ ]:
X_test_submission.shape

In [ ]:
# Aplicando o scaling

In [ ]:
#X_train1 = X_train_bal

In [ ]:
#importando a biblioteca
#from sklearn.preprocessing import StandardScaler
#instanciando a variável
#sc = StandardScaler()
#ajustando os dados de treino
#X_train1 = sc.fit_transform(X_train1)
#transformando os dados de teste
X_test_submission = sc.transform(X_test_submission)

In [ ]:
X_test_submission

In [ ]:
X_test_submission.shape

In [ ]:
# Aplicando o pca.transform em X_test_submission para redução da dimensionalidade

In [ ]:
X_test_submission = pca.transform(X_test_submission)

In [ ]:
X_test_submission

In [ ]:
X_test_submission.shape

In [ ]:
#Agora temos 50 features em vez de 200

In [ ]:
# Calculando a probabilidade y predito=1 com a LogisticRegression

In [ ]:
logreg 

In [ ]:
y_pred_logreg = logreg.predict_proba(X_test_submission)

In [ ]:
y_pred_logreg

In [ ]:
y_pred_logreg = y_pred_logreg[:,:1]

In [ ]:
y_pred_logreg

In [ ]:
y_pred_logreg.shape

In [ ]:
df_logReg= df_sample.copy()

In [ ]:
df_logReg.head()

In [ ]:
type(df_logReg)

In [ ]:
df_logReg = df_logReg.drop(columns = ['target'])

In [ ]:
df_logReg['target']=y_pred_logreg

In [ ]:
df_logReg.head() 

In [ ]:
df_logReg.to_csv('resultado_RegLog_04_10_pred_prob.csv', index=False)

In [ ]:
teste = pd.read_csv('resultado_RegLog_04_10_pred_prob.csv')
teste.head()

## Resultado do Kaggle

In [ ]:
# Após submissão do arquivo no Kaggle do modelo com Regressão Logística:

In [ ]:
# Private Score: 0.85786
# Public Score: 0.85806